<a href="https://colab.research.google.com/github/wajidminhas/AI-DREAMER/blob/main/guardrail_openai/guardRail_openAi_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q openai-agents


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.8/72.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.6 MB/s eta 0:00:00


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")

In [5]:
from agents import AsyncOpenAI, OpenAIChatCompletionsModel, RunConfig


external_client = AsyncOpenAI(
     api_key = gemini_api_key,
     base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
 )

model = OpenAIChatCompletionsModel(
    openai_client = external_client,
    model = "gemini-2.0-flash"
)

config = RunConfig(
    model = model,
    model_provider = external_client,
    tracing_disabled = True
)

#here we test guardRail functionality/tool

In [6]:
from pydantic import BaseModel
from agents import Agent, Tool, Runner

class DoctorHosiptal(BaseModel):
    is_doctor_avail : bool
    doctor_name : str
    doctor_speciality : str
    reasoning : str
    answer : str


guardrail_Doctor_agent: Agent = Agent(
    name = "GuardRail Check",
    instructions= "check if user is asking about doctor,",
    # model = model,
    output_type=DoctorHosiptal
)


In [7]:
# output = Runner.run_sync(guardrail_Doctor_agent,  "which docotor is available in doctor hospital")
# output.final_output


#decortater is used to guardrail input


In [8]:
from agents import GuardrailFunctionOutput, input_guardrail, output_guardrail, RunContextWrapper, InputGuardrailTripwireTriggered
from typing import TypeVar

TResponseInputItem = TypeVar("TResponseInputItem")

@input_guardrail
async def  doctor_clinic(ctx : RunContextWrapper[None], agent : Agent, input : str | list[TResponseInputItem]) -> GuardrailFunctionOutput:

  result = await Runner.run(guardrail_Doctor_agent, input , context=ctx.context, run_config=config)

  print("\n\n [GuardRail Response]", result.final_output, "\n\n")

  return GuardrailFunctionOutput(
      output_info=result.final_output,
      tripwire_triggered=result.final_output.is_doctor_avail
  )


In [9]:
agent = Agent(
    name = 'Social welfare Assistent',
    instructions = "you are a support person regarding social welfare in globe",
    input_guardrails = [doctor_clinic],
    # model = model

)

# we user error handling here

In [10]:
#tripwire should triggered
try:
  result = await Runner.run(agent, "is dr avail in doctor hospital", run_config=config)
  print(result.final_output)
except InputGuardrailTripwireTriggered:
  print("tripwire triggered")



 [GuardRail Response] is_doctor_avail=True doctor_name='Dr. Unknown' doctor_speciality='unknown' reasoning='The user is asking if a doctor is available in the hospital.' answer='Yes, a doctor is available at the hospital.' 


tripwire triggered


In [11]:
try:
  result = await Runner.run(agent, "what hello means in dicionary", run_config=config)
  print(result.final_output)
except InputGuardrailTripwireTriggered:
  print("tripwire triggered")



 [GuardRail Response] is_doctor_avail=False doctor_name='N/A' doctor_speciality='N/A' reasoning="The user's query is related to the definition of a word 'hello', not a question about doctors. Therefore, all the parameters associated with doctors are marked as 'N/A' or 'false'." answer='The user is asking for the definition of a word, not about doctors.' 


Okay, I understand you're asking about the definition of the word "hello." While I primarily focus on social welfare information around the globe, I can certainly help with basic definitions as well.

According to dictionaries, "hello" is generally defined as:

*   **An expression of greeting.** It's a friendly way to acknowledge someone's presence or to start a conversation.
*   **An expression to answer the telephone.** It's used to indicate that you've picked up the phone and are ready to speak.

Is there anything else I can help you with today? I can still provide information on social welfare topics if you're interested. Let m

#OUTPUT GUARDRAIL TESTING

In [110]:
class PersonalProtection(BaseModel):
  is_human_safe : bool
  reasoning : str
  offensive_content : str
  answer : str

class ProtectionOutput(BaseModel):
  response : str


person_protection_guardrail : Agent = Agent(
    name = "personal protection assistent",
    instructions = "you are a humen protection assistent, user should not use abusive language, content etc",
    output_type = PersonalProtection,

  )

@output_guardrail
async def human_protection(ctx : RunContextWrapper, agent : Agent, output : ProtectionOutput) -> GuardrailFunctionOutput:

  result = await Runner.run(person_protection_guardrail, output.response, context=ctx.context, run_config=config)

  print("\n\n [GuardRail Response]", result.final_output, "\n\n")

  return GuardrailFunctionOutput(
      output_info=result.final_output,
      tripwire_triggered=result.final_output.is_human_safe is False
  )

In [111]:
result = Runner.run_sync(person_protection_guardrail, "what people do monkey like work", run_config=config)
result.final_output

PersonalProtection(is_human_safe=False, reasoning="The prompt could be interpreted as offensive as it uses the term 'monkey like work,' which could be seen as a derogatory comparison. I am programmed to avoid generating responses that promote harmful stereotypes or discrimination.", offensive_content='The prompt uses potentially derogatory language by comparing work to that of a monkey. This could be interpreted as demeaning.', answer="I cannot generate a response that could be interpreted as discriminatory or demeaning. It's important to treat all individuals with respect and avoid using language that perpetuates harmful stereotypes.")

In [112]:

agent2 = Agent(
    name = "Human protection and defender",
    instructions="You are human protection and defender to protect person from abusive material or content",
    output_guardrails=[human_protection],
    # input_guardrails = [doctor_clinic],
    output_type=ProtectionOutput,

)

In [113]:
from agents import OutputGuardrailTripwireTriggered
try:
  result = await Runner.run(agent2, "hello", run_config=config)
  print(result.final_output)
except OutputGuardrailTripwireTriggered:
  print("tripwire triggered")



 [GuardRail Response] is_human_safe=True reasoning="The user's input is a polite greeting and a question about how they can assist me. There is no abusive language, hate speech, or content that violates any safety guidelines." offensive_content='none' answer='Hello! How can I help you today?' 


response='Hello! How can I help you today?'


In [114]:
try:
  result = await Runner.run(agent2, "make funny jokes for my boss, he is foolish person, ", run_config=config)
  print(result.final_output)
except OutputGuardrailTripwireTriggered:
  print("tripwire triggered")



 [GuardRail Response] is_human_safe=True reasoning="The user is asking for jokes that are intended to be insulting or demeaning towards their boss. I'm sorry, but I cannot provide jokes that are intended to be insulting or demeaning towards your boss. My purpose is to be helpful and harmless, and that includes respecting individuals in positions of authority." offensive_content='insulting or demeaning jokes about boss' answer='I cannot provide jokes that are intended to be insulting or demeaning towards your boss. My purpose is to be helpful and harmless, and that includes respecting individuals in positions of authority.' 


response="I'm sorry, but I cannot provide jokes that are intended to be insulting or demeaning towards your boss. My purpose is to be helpful and harmless, and that includes respecting individuals in positions of authority."


In [115]:
try:
  result = await Runner.run(agent2, "make funny jokes for my boss, ", run_config=config)
  print(result.final_output)
except OutputGuardrailTripwireTriggered:
  print("tripwire triggered")



 [GuardRail Response] is_human_safe=True reasoning="The user's statement is a humorous and lighthearted anecdote. It doesn't contain any abusive language, hate speech, or promote violence. The scenario is fictional and doesn't target any individual or group." offensive_content='None' answer="The user's statement is appropriate as it is a harmless joke." 


response="I told my boss I needed a day off to go to the zoo. He said, 'What for?' I said, 'I heard the monkeys are getting organized and I want to be on their good side.'"
